In [1]:
%load_ext autoreload
%autoreload 2 

In [2]:
from fastai.tabular.all import * 
from mock import Mock
from tabnet.utils import *
from tabnet.model import *

In [46]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [63]:
class LinDecoder(Module):
    def __init__(self, n_cat, n_cont, ps=0.1, n_steps=5, n_d=12, **kwargs):
        store_attr()
        set_trace()
        self.decoder = nn.Sequential(
            LinBnDrop(n_d, 256, p=ps, act=Mish()),
            LinBnDrop(256, 512, p=ps, act=Mish()),
            LinBnDrop(512, 1024, p=ps, act=Mish())
        )
        
        self.decoder_cont = nn.Sequential(
            LinBnDrop(1024, n_cont, p=ps, bn=False, act=None),
        )
        
        self.decoder_cat = LinBnDrop(1024, n_cat, p=ps, bn=False, act=None)

        
    def forward(self, x):
        xs = x.sum(dim=1)
        decoded = self.decoder(xs)
        x_cat = self.decoder_cat(decoded)
        x_cont = self.decoder_cont(decoded)
        
        return torch.cat([x_cat, x_cont], dim=1)

In [62]:
def lin_decoder(n_cat, n_cont, **kwargs):
    return LinDecoder(n_cat, n_cont, **kwargs)

In [64]:
LinDecoder(10, 3,3, n_steps=5, n_steps=5)

SyntaxError: keyword argument repeated (<ipython-input-64-630c49ce2593>, line 1)

# Tests

### SS Forest

In [39]:
data_dir = Path('./data')

In [48]:
def extract_gzip(file, dest=None):
    import gzip
    dest = dest or Path(dest)
    with gzip.open(file, 'rb') as f_in:
        with open(dest / file.stem, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

In [49]:
forest_type_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz'
forest_path = untar_data(forest_type_url, dest=data_dir, extract_func=extract_gzip)

In [55]:
target = "Covertype"

cat_names = [
    "Wilderness_Area1", "Wilderness_Area2", "Wilderness_Area3",
    "Wilderness_Area4", "Soil_Type1", "Soil_Type2", "Soil_Type3", "Soil_Type4",
    "Soil_Type5", "Soil_Type6", "Soil_Type7", "Soil_Type8", "Soil_Type9",
    "Soil_Type10", "Soil_Type11", "Soil_Type12", "Soil_Type13", "Soil_Type14",
    "Soil_Type15", "Soil_Type16", "Soil_Type17", "Soil_Type18", "Soil_Type19",
    "Soil_Type20", "Soil_Type21", "Soil_Type22", "Soil_Type23", "Soil_Type24",
    "Soil_Type25", "Soil_Type26", "Soil_Type27", "Soil_Type28", "Soil_Type29",
    "Soil_Type30", "Soil_Type31", "Soil_Type32", "Soil_Type33", "Soil_Type34",
    "Soil_Type35", "Soil_Type36", "Soil_Type37", "Soil_Type38", "Soil_Type39",
    "Soil_Type40"
]

cont_names = [
    "Elevation", "Aspect", "Slope", "Horizontal_Distance_To_Hydrology",
    "Vertical_Distance_To_Hydrology", "Horizontal_Distance_To_Roadways",
    "Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm",
    "Horizontal_Distance_To_Fire_Points"
]

feature_columns = (
    cont_names + cat_names + [target])

params = dict(cont_names = cont_names, y_names = target, cat_names = cat_names)
procs=[Categorify, FillMissing, Normalize]
model_params = dict(n_d=64, n_a=64, n_steps=5, virtual_batch_size=512, gamma=1.5, bs=1024*16,
                    lambda_sparse=1e-4, momentum=0.7, n_shared_ft_blocks=2, n_independent_ft_blocks=2, p=0.5)

In [56]:
df = pd.read_csv(forest_path, header=None, names=feature_columns).sample(n=200_000)
df.shape

(200000, 55)

In [57]:
val_pct = 0.2
curriculum = False
head = lambda n_out, **kwargs: LinDecoder(n_out, **kwargs)
loss_func = MSELossFlat()

In [58]:
to = tabular_pandas(df, cat_names, cont_names, target, tabular_type=TabularPandasIdentity, val_pct=val_pct)
dls = to.dataloaders(bs=model_params['bs'])
dls.n_inp = 2
cbs = [SetPrior(), TabularMasking(p=model_params['p'], curriculum=curriculum),
       MaskRegularizer(model_params['lambda_sparse'])]
model = TabNetSelfSupervised(head, to, **model_params)

> <ipython-input-47-2cea77a34166>(5)__init__()
-> self.decoder = nn.Sequential(
(Pdb) c


In [59]:
learn = Learner(dls, model, cbs=cbs, loss_func=loss_func, metrics=[mse])

In [60]:
learn.lr_find()

/opt/conda/lib/python3.7/site-packages/torch/nn/modules/loss.py:445: UserWarning: Using a target size (torch.Size([884736])) that is different to the input size (torch.Size([65536])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (65536) must match the size of tensor b (884736) at non-singleton dimension 0

### SS Flowers

In [5]:
data = load_iris()
X,y = data['data'], data['target']
cont_names = ['s_len', 's_wid', 'p_len', 'p_wid']
cat_names = []
y_names = 'target'
df = pd.concat([pd.DataFrame(X, columns=cont_names),
           pd.DataFrame(y, columns=[y_names])], axis=1)

In [27]:
model_params = dict(n_d=16, n_a=16, n_steps=3, virtual_batch_size=4, gamma=1.5, bs=32,
                    lambda_sparse=1e-4, momentum=0.7, n_shared_ft_blocks=2, n_independent_ft_blocks=0, p=0.3)
val_pct = 0.2
curriculum = False
head = lambda n_out, **kwargs: LinDecoder(n_out, **kwargs)
loss_func = MSELossFlat()

In [28]:
to = tabular_pandas(df, cat_names, cont_names, y_names, tabular_type=TabularPandasIdentity, val_pct=val_pct)
dls = to.dataloaders(bs=model_params['bs'])
dls.n_inp = 2
cbs = [SetPrior(), TabularMasking(p=model_params['p'], curriculum=curriculum),
       MaskRegularizer(model_params['lambda_sparse'])]
model = TabNetSelfSupervised(head, to, **model_params)

> <ipython-input-4-2cea77a34166>(5)__init__()
-> self.decoder = nn.Sequential(
(Pdb) c


In [29]:
learn = Learner(dls, model, cbs=cbs, loss_func=loss_func, metrics=[mse])

In [31]:
learn.dls.cpu()

In [32]:
learn.fit_one_cycle(30, slice(1e-4, 1e-3))

epoch,train_loss,valid_loss,mse,time
0,1.040652,1.091283,1.091248,00:00
1,0.951325,1.034003,1.033969,00:00
2,0.859640,0.962954,0.962906,00:00
3,0.833971,0.729722,0.729674,00:00
4,0.797346,0.858728,0.858683,00:00
5,0.730077,0.828966,0.828915,00:00
6,0.728487,0.717524,0.717481,00:00
7,0.710917,0.733799,0.733735,00:00
8,0.697380,1.138567,1.138513,00:00
9,0.686746,0.584651,0.584596,00:00


In [33]:
dl = learn.dls.test_dl(df)
preds, targs = learn.get_preds(dl=dl)

In [34]:
preds[0][:10], targs[0][:10]

(tensor([-0.9898,  0.5815, -1.0864, -0.9678]),
 tensor([-0.8820,  1.0085, -1.3390, -1.2946]))

### BaseLine

In [ ]:
splits = RandomSplitter()(range_of(df))
dls = TabularPandas(df, cat_names=[], cont_names=['s_len', 's_wid', 'p_len', 'p_wid'], splits=splits,
              procs=[Categorify, FillMissing, Normalize], y_names='target', y_block=CategoryBlock()).dataloaders(bs=4)

In [ ]:
learn = tabular_learner(dls, metrics=[accuracy])

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(20, 1e-2)

### TabNet

In [ ]:
model_params = dict(n_d=16, n_a=16, n_steps=5, virtual_batch_size=512, gamma=1.5, bs=30,
                    lambda_sparse=1e-4, momentum=0.7, n_shared_ft_blocks=2, n_independent_ft_blocks=2)

In [ ]:
learn = tabnet_df_classifier(df, cat_names=[], cont_names=['s_len', 's_wid', 'p_len', 'p_wid'],
                                                     y_names='target', val_pct=0.2, **model_params)

In [ ]:
learn.summary()

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(10, 1e-2)

# Export

In [ ]:
from nbdev.export import notebook2script
notebook2script()